# Tiger Tutorial: Solving POMDPs

This tutorial outlines how to define a POMDP using the [POMDPs.jl](https://github.com/sisl/POMDPs.jl) interface. We will go through a simple problem simply known as the tiger problem (we will refer to it as the tiger POMDP). After defining the tiger POMDP, we will use QMDP and SARSOP to solve the POMDP. If you are new to working with this package, check out the [tutorial](http://nbviewer.ipython.org/github/sisl/POMDPs.jl/blob/master/examples/GridWorld.ipynb) on MDPs first.

## Dependencies
You need to install a few modules in order to use this notebook. If you have all the modules below installed, great! If not run the following commands:

```julia
# install the POMDPs.jl interface
Pkg.clone("https://github.com/sisl/POMDPs.jl.git")

# install the SARSOP wrapper
Pkg.clone("https://github.com/sisl/SARSOP.jl")
# build SARSOP, it builds from source, so this may take some time
Pkg.build("SARSOP")

# install the QMDP solver
Pkg.clone("https://github.com/sisl/QMDP.jl")

# install two helper modules
Pkg.clone("https://github.com/sisl/POMDPToolbox.jl") # this provides implementations of discrete belief updating
Pkg.clone("https://github.com/sisl/POMDPDistributions.jl") # helps with sampling 
```

If you already have all of the modules above, make sure you have the most recent versions. Many of these are still under heavy development, so update before starting by running

```julia
Pkg.update()
```

## Problem Overview
In the tiger POMDP, the agent is tasked with escaping from a room. There are two doors leading out of the room. Behind one of the doors is a tiger, and behind the other is sweet, sweet freedom. If the agent opens the door and finds the tiger, it gets eaten (and receives a reward of -100). If the agent opens the other door, it escapes and receives a reward of 10. The agent can also listen. Listening gives a noisy measuremnt of which door the tiger is hiding behind. Listening gives the agent the correct location of the tiger 85% of the time. The agent receives a reward of -1 for listening. 

In [1]:
# first import POMDPs.jl
using POMDPs

## POMDP 
A POMDP is defined by the tuple
$$(\mathcal{S}, \mathcal{A}, \mathcal{Z}, T, R, O).$$
In addition to the familiar, state $\mathcal{S}$ and action $\mathcal{A}$ spaces, we must also define an observation space $\mathcal{Z}$ and an observation function $O$. The POMDP problem definition may be similar to the one for MDP. For example, if you wanted to add state uncertaitniy to your problem, you can define the observation space, and observation function in addition to your previous MDP definition.

Before defining the spaces for this problem, let's first deinfe the concrete type for the tiger POMDP. 

In [2]:
type TigerPOMDP <: POMDP
    r_listen::Float64 # reward for listening (default -1)
    r_findtiger::Float64 # reward for finding the tiger (default -100)
    r_escapetiger::Float64 # reward for escaping (default 10)
    p_listen_correctly::Float64 # prob of correctly listening (default 0.85)
    discount_factor::Float64 # discount
end
# default constructor
function TigerPOMDP()
    return TigerPOMDP(-1.0, -100.0, 10.0, 0.85, 0.95)
end

TigerPOMDP

There are a number of parameters in the problem definition, but we can treat them all as constants. You can read more about the Tiger problem and POMDPs [here](http://www.techfak.uni-bielefeld.de/~skopp/Lehre/STdKI_SS10/POMDP_tutorial.pdf#page=28). However, we created a default constructor that allows us to initialize the tiger POMDP by simply running:

In [3]:
pomdp = TigerPOMDP()

TigerPOMDP(-1.0,-100.0,10.0,0.85,0.95)

## States
We define a concrete type to represent the state of the tiger POMDP. The only thing we need to know to represent the problem is which door the tiger is hiding behind. 

In [4]:
type TigerState <: State
    tigerleft::Bool
end
# initialization function
POMDPs.create_state(::TigerPOMDP) = TigerState(true);

Since the state is a binary value, we represent it as a boolean, but we could have represented it as an integer or any other sensible type.

## Actions
There are three possible actions our agent can take: open the left door, open the right door, and listen. For clarity, we will represent these with symbols.

In [5]:
type TigerAction <: Action
    act::Symbol
end
# initialization function
POMDPs.create_action(::TigerPOMDP) = TigerAction(:listen);

We will represent our actions with the following symbols: open left (:openl), open right (:openr), and listen (:listen). For example, the action below represnts listening:

In [6]:
a = TigerAction(:listen)

TigerAction(:listen)

## Observations
There are two possible observations: the agent either hears the tiger behind the left door or behind the right door. We use a boolean to represent the observations:

In [7]:
type TigerObservation <: Observation
    obsleft::Bool 
end
# initialization function
POMDPs.create_observation(::TigerPOMDP) = TigerObservation(true);

## Spaces
Let's define our state, action and observation spaces.

### State Space
There are only two states in the tiger POMDP: the tiger is either behind the left door or behind the right door. Our state space is simply an array of the states in the problem.

In [8]:
type TigerStateSpace <: AbstractSpace
    states::Vector{TigerState} 
end

Let's now define the ```states``` and the ```iterator``` functions. Recall, that the ```states``` function returns the state space for a given POMDP type, and the ```iterator``` function returns an iterator for a given space. Here, the iterator function returns an array of the two possible states in our problem.

In [9]:
POMDPs.states(::TigerPOMDP) = TigerStateSpace([TigerState(true), TigerState(false)])
POMDPs.iterator(space::TigerStateSpace) = space.states;
POMDPs.index(::TigerPOMDP, s::TigerState) = (Int64(s.tigerleft) + 1)

index (generic function with 3 methods)

Recall that in the [tutorial](http://nbviewer.ipython.org/github/sisl/POMDPs.jl/blob/master/examples/GridWorld.ipynb) on MDPs, we also defined a ```rand!``` function that sampled the space. We do not need this function when using QMDP or SARSOP. However, if you wanted to use Monte Carlo solvers solvers like POMCP or DESPOT you would need a function that can sample your spaces. You should also create these sampling functions if you plan to simulate your policy.

### Action Space
There are three actions in our problem. Once again, we represent the action space as an array of the actions in our problem. The ```actions``` and ```iterator``` functions serve a similar purpose to the ```states``` and ```iterator``` functions above.

In [10]:
type TigerActionSpace <: AbstractSpace
    actions::Vector{TigerAction} 
end
# define actions function
POMDPs.actions(::TigerPOMDP) = TigerActionSpace([TigerAction(:openl), TigerAction(:openr), TigerAction(:listen)]); # default
POMDPs.actions(::TigerPOMDP, ::TigerState, acts::TigerActionSpace) = acts; # convenience (actions do not change in different states)
# define iterator function
POMDPs.iterator(space::TigerActionSpace) = space.actions;

### Observation Space
The observation space looks similar to the state space. Recall that the state represents the truth about our system, while the observation is potentially untuthful information recieves about the state. In the tiger POMDP, our observation could give us a false representation of our state. 

In [11]:
type TigerObservationSpace <: AbstractSpace
    obs::Vector{TigerObservation} 
end
# function returning observation space
POMDPs.observations(::TigerPOMDP) = TigerObservationSpace([TigerObservation(true), TigerObservation(false)]);
POMDPs.observations(::TigerPOMDP, s::TigerState, obs::TigerObservationSpace) = obs;
# function returning an iterator over that space
POMDPs.iterator(space::TigerObservationSpace) = space.obs;

Now that we've defined the POMDP spaces, let's move on to defining the model functions.

## Transition and Observation Distributions
Before defining the model functions, we first need to create a distributions data-type. In general, our distributions should support sampling and have a ```pdf``` method. If you only want to get a policy from the SARSOP and QMDP solvers, you do not need to worry about implementing a sampling function. However, if you want to simulate the policy, you should implement these functions.

Since the transition and observation distributions have identical form, we could just use a single type to serve the needs of both. This will not be the case in general, and we will define two seperate types (although similar looking) for clarity. 

In [12]:
# transition distribution type
type TigerTransitionDistribution <: AbstractDistribution
    probs::Vector{Float64} 
end
# transition distribution initializer
POMDPs.create_transition_distribution(::TigerPOMDP) = TigerTransitionDistribution([0.5, 0.5])

# observation distribution type
type TigerObservationDistribution <: AbstractDistribution
    probs::Vector{Float64} 
end
# observation distribution initializer
POMDPs.create_observation_distribution(::TigerPOMDP) = TigerObservationDistribution([0.5, 0.5]);

We now define the ```pdf``` function. For a discrete problem, this function returns the probability mass of a given element (state or observation in our case).

In [13]:
# transition pdf
function POMDPs.pdf(d::TigerTransitionDistribution, s::TigerState)
    s.tigerleft ? (return d.probs[1]) : (return d.probs[2]) 
end;
# obsevation pdf
function POMDPs.pdf(d::TigerObservationDistribution, o::TigerObservation)
    o.obsleft ? (return d.probs[1]) : (return d.probs[2])
end;

Finally, let's create the sampling functions. We will use [POMDPDistributions.jl](https://github.com/sisl/POMDPDistributions.jl) to help us implement the sampling function. However, you are free to use any approach you want.

In [14]:
using POMDPDistributions

# sample from transition distribution
function POMDPs.rand(rng::AbstractRNG, d::TigerTransitionDistribution, s::TigerState)
    # we use a categorical distribution, and this will usually be enough for a discrete problem
    c = Categorical(d.probs) # this comes from POMDPDistributions
    # sample an integer from c
    sp = rand(rng, c) # this function is also from POMDPDistributions
    # if sp is 1 then tiger is on the left
    sp == 1 ? (s.tigerleft=true) : (s.tigerleft=false)
    return s
end

# similar for smapling from the observation distribution
function POMDPs.rand(rng::AbstractRNG, d::TigerObservationDistribution, o::TigerObservation)
    c = Categorical(d.probs) 
    op = rand(rng, c) 
    # if op is 1 then we hear tiger on the left
    op == 1 ? (o.obsleft=true) : (o.obsleft=false)
    return o
end;

This is all we have to do for our distribution functions!

## Transition Model
Here we define the transition model for the tiger POMDP. The model itself is fairly simple. Our state is represented by the location of the tiger (left or right). The location of the tiger doesn't change when the agent listens. However, after the agent opens the door, it reaches a terminal state. That is the agent either escapes or gets eaten. To simplify our formulation, we simply reset the location of the tiger randomly. We could model this problem with a terminal state (i.e. one in which the agent no longer receives reward) as well. 

In [15]:
# the transition mode
function POMDPs.transition(pomdp::TigerPOMDP, s::TigerState, a::TigerAction, d::TigerTransitionDistribution=create_transition_distribution(pomdp))
    probs = d.probs
    # if open a door reset the tiger probs
    if a.act == :openl || a.act == :openr
        fill!(probs, 0.5)
    # if tiger is on the left, distribution = 1.0 over the first state
    elseif s.tigerleft
        probs[1] = 1.0
        probs[2] = 0.0
    # otherwise distribution = 1.0 over the second state
    else
        probs[1] = 0.0
        probs[2] = 1.0
    end
    d
end;

## Reward Model
The reward model caputres the immediate objectives of the agent. It recieves a large negative reward for opening the door with the tiger behind it (-100), gets a positive reward for opening the other door (+10), and a small penalty for listening (-1).

In [16]:
function POMDPs.reward(pomdp::TigerPOMDP, s::TigerState, a::TigerAction)
    r = 0.0
    # small penalty for listening
    if a.act == :listen
        r += pomdp.r_listen
    end
    if a.act == :openl
        # find tiger behind left door
        if s.tigerleft
            r += pomdp.r_findtiger
        # escape through left door
        else
            r += pomdp.r_escapetiger
        end
    end
    if a.act == :openr
        # escape through the right door
        if s.tigerleft
            r += pomdp.r_escapetiger
            # find tiger behind the right door
        else
            r += pomdp.r_findtiger
        end
    end
    return r
end;
# to match the interface
POMDPs.reward(pomdp::TigerPOMDP, s::TigerState, a::TigerAction, sp::TigerState) = reward(pomdp, s, a)

reward (generic function with 3 methods)

## Observation Model
The observation model captures the uncertaintiy in the agent's lsitening ability. When we listen, we receive a noisy measurement of the tiger's location. 

In [17]:
function POMDPs.observation(pomdp::TigerPOMDP, s::TigerState, a::TigerAction, d::TigerObservationDistribution=create_observation_distribution(pomdp))
    probs = d.probs
    p = pomdp.p_listen_correctly # probability of listening correctly
    if a.act == :listen
        # if tiger is behind left door
        if s.tigerleft
            probs[1] = p # correct prob
            probs[2] = (1.0-p) # wring prob
        # if tiger is behind right door
        else
            probs[1] = (1.0-p) # wrong prob
            probs[2] = p # correct prob
        end
    # if don't listen uniform
    else
        fill!(probs, 0.5)
    end
    d
end;

## Miscallenous Functions

Let's define the ```discount``` function and the functions that return the size of our spaces.

In [18]:
POMDPs.discount(pomdp::TigerPOMDP) = pomdp.discount_factor
POMDPs.n_states(::TigerPOMDP) = 2
POMDPs.n_actions(::TigerPOMDP) = 3
POMDPs.n_observations(::TigerPOMDP) = 2;

## Beliefs
If you are somewhat familiar with Julia defining all of the above may have been relaitvely simple. However, all POMDPs must be represented with a belief. Implementing beliefs and their updaters can be tricky. Luckily, we provide you with some nice tools to work with beliefs. Note that if you just want to use SASROP to solve for the alpha-vectors, and use your own belief updating scheme you do not need to implement the functions below.

We will use the [POMDPToolbox](https://github.com/sisl/POMDPToolbox.jl) module which implements a discrete belief type and a belief udpater for it. You can create a ```DiscreteBelief``` type by passing in the size of your state space. This will create a belief with an initial uniform distribution over the states. To update the belief, you will need to create a ```DiscreteUpdater``` by passing your model POMDP into it. You can see how this is done in the simulation loop below. 

In [19]:
# we will use the POMDPToolbox module
using POMDPToolbox

# define a initialization function
POMDPs.create_belief(::TigerPOMDP) = DiscreteBelief(2) # the belief is over our two states
# initial belief is same as create
POMDPs.initial_belief(::TigerPOMDP) = DiscreteBelief(2);


## SARSOP Solver
Let's play around with the [SARSOP.jl](https://github.com/sisl/SARSOP.jl) solver. The module we provide is a wrapper for the SARSOP backend. You can find more information about it [here](http://bigbird.comp.nus.edu.sg/pmwiki/farm/appl/).

In [22]:
using SARSOP # load the module
# initialize our tiger POMDP
pomdp = TigerPOMDP()

# what follows are functions provided by SARSOP
policy = POMDPPolicy("tiger.policy") # initialize the policy, the argument is the name you want for your policy file 
# create the .pomdpx file, this is the format which the SARSOP backend reads in
pomdpfile = POMDPFile(pomdp, "tiger.pomdpx") # must end in .pomdpx
# initialize the solver
solver = SARSOPSolver()
# run the solve function
solve(solver, pomdpfile, policy)

Generating a pomdpx file: tiger.pomdpx


POMDPXFile.POMDPAlphas(2x5 Array{Float64,2}:
 -81.5975   3.01448  24.6954    28.4025  19.3711
  28.4025  24.6954    3.01452  -81.5975  19.3711,[0,2,2,1,2])


Loading the model ...
  input file   : tiger.pomdpx
  loading time : 0.00s 

SARSOP initializing ...
  initialization time : 0.00s

-------------------------------------------------------------------------------
 Time   |#Trial |#Backup |LBound    |UBound    |Precision  |#Alphas |#Beliefs  
-------------------------------------------------------------------------------
 0       0       0        -20        92.8206    112.821     3        1        
 0       2       51       -6.2981    63.1396    69.4377     7        16       
 0       4       103      0.149651   52.2764    52.1268     9        21       
 0       6       151      6.19248    42.0546    35.8621     9        21       
 0       8       200      10.3563    35.232     24.8757     12       21       
 0       11      250      14.0433    29.5471    15.5037     6        21       
 0       14      300      16.545     25.0926    8.54759     10       21       
 0.01    17      350      18.2281    21.8163    3.5882      14       21   

In [23]:
# we can retrieve the alpha vectors by calling
alphas(policy)

2x5 Array{Float64,2}:
 -81.5975   3.01448  24.6954    28.4025  19.3711
  28.4025  24.6954    3.01452  -81.5975  19.3711

Let's now see how our policy changes with the belief.

In [24]:
# let's initialize the beliefs
b = initial_belief(pomdp) # the initial prior

POMDPToolbox.DiscreteBelief([0.5,0.5],[0.5,0.5],2,true)

In [25]:
ai = action(policy, b) # index of action, you need to convert this to the true action, support for automatic conversion is coming soon
# the index corresponds to the action in our action array
action_map = iterator(actions(pomdp)) # create a mapping array
a = action_map[ai] # get the actions

TigerAction(:listen)

Given our uniform prior, the optimal policy is to listen. Let's now update our belief. To do that, we need to sample an observation from the system state. Recall that a belief update requires, an action, an observation, and the prior belief. Let's make a small simulation loop to see what happens to our system.

In [26]:
s = create_state(pomdp)
o = create_observation(pomdp)

b = initial_belief(pomdp)

updater = DiscreteUpdater(pomdp) # this comes from POMDPToolbox

rng = MersenneTwister(9) # initialize a random number generator

rtot = 0.0
# lets run the simulation for ten time steps
for i = 1:10
    # get the action from our SARSOP policy
    ai = action(policy, b)
    a = action_map[ai]
    # compute the reward
    r = reward(pomdp, s, a)
    rtot += r
    
    println("Time step $i")
    println("Have belief: $(b.b), taking action: $(a), got reward: $(r)")
    
    # transition the system state
    trans_dist = transition(pomdp, s, a)
    rand(rng, trans_dist, s)

    # sample a new observation
    obs_dist = observation(pomdp, s, a)
    rand(rng, obs_dist, o)
    
    # update the belief
    b = update(updater, b, a, o)
    
    println("Saw observation: $(o), new belief: $(b.b)\n")

end
println("Total reward: $rtot")

Time step 1
Have belief: [0.5,0.5], taking action: TigerAction(:listen), got reward: -1.0
Saw observation: TigerObservation(true), new belief: [0.85,0.15000000000000002]

Time step 2
Have belief: [0.85,0.15000000000000002], taking action: TigerAction(:listen), got reward: -1.0
Saw observation: TigerObservation(true), new belief: [0.9697986577181208,0.030201342281879207]

Time step 3
Have belief: [0.9697986577181208,0.030201342281879207], taking action: TigerAction(:openr), got reward: 10.0
Saw observation: TigerObservation(false), new belief: [0.5,0.5]

Time step 4
Have belief: [0.5,0.5], taking action: TigerAction(:listen), got reward: -1.0
Saw observation: TigerObservation(false), new belief: [0.15000000000000002,0.85]

Time step 5
Have belief: [0.15000000000000002,0.85], taking action: TigerAction(:listen), got reward: -1.0
Saw observation: TigerObservation(false), new belief: [0.030201342281879207,0.9697986577181208]

Time step 6
Have belief: [0.030201342281879207,0.969798657718120

Notice that over the first six time steps, the policy is fairly simple. We listen twice, and then decide which door to open. However, in the following steps, we get a mix of observations, which makes the decision harder. Our agent does not open a door, because its belief is still uniform at the last time step! 

## QMDP Solver
We will briefly go over the [QMDP.jl](https://github.com/sisl/QMDP.jl) solver. You should use QMDP with a word of caution. QMDP assumes that all state uncetainty dissapears in the next time step. This could lead to bad policies in problems with information gathering actions. For example, in the tiger POMDP listening is an information gathering action, and the resulting QMDP policy is quite poor. However, QMDP can work very well in problems where the state uncertainity is not impacted by the agent's action (for example systems with noisy sensor measurements).

In [27]:
using QMDP

# initialize the solver
# key-word args are the maximum number of iterations the solver will run for, and the Bellman tolerance
solver = QMDPSolver(max_iterations=20, tolerance=1e-3) 

# initialize the QMDP policy
qmdp_policy = create_policy(solver, pomdp)

# run the solver
solve(solver, pomdp, qmdp_policy, verbose=true)

Iteration : 1, residual: 14.75, iteration run-time: 1.9473e-5, total run-time: 1.9473e-5


QMDP.QMDPPolicy(2x3 Array{Float64,2}:
  37.6943  147.694   137.586
 149.448    41.1425  139.31 ,POMDPs.Action[TigerAction(:openl),TigerAction(:openr),TigerAction(:listen)])

Iteration : 2, residual: 12.59046875, iteration run-time: 1.0315e-5, total run-time: 2.9788e-5
Iteration : 3, residual: 11.564691406249999, iteration run-time: 4.968e-6, total run-time: 3.4756e-5
Iteration : 4, residual: 10.943236428222654, iteration run-time: 7.282e-6, total run-time: 4.2038e-5
Iteration : 5, residual: 10.2558588273941, iteration run-time: 6.729e-6, total run-time: 4.8767e-5
Iteration : 6, residual: 9.587976314837448, iteration run-time: 4.085e-6, total run-time: 5.2852e-5
Iteration : 7, residual: 8.957886507199987, iteration run-time: 3.764e-6, total run-time: 5.6616e-5
Iteration : 8, residual: 8.367828168991792, iteration run-time: 3.985e-6, total run-time: 6.0601e-5
Iteration : 9, residual: 7.816304847983972, iteration run-time: 3.791e-6, total run-time: 6.4392e-5
Iteration : 10, residual: 7.301052156282381, iteration run-time: 3.728e-6, total run-time: 6.812e-5
Iteration : 11, residual: 6.8197456599030915, iteration run-time: 3.742e-6, total run-time: 7.1862000000

Notice that these alpha-vectors differ from those compute by SARSOP. Let's see how the policy looks in simulation. The simulation loop is the same as we saw above, but now we use the QMDP policy instead of the SARSOP one.

In [28]:
s = create_state(pomdp)
o = create_observation(pomdp)

b = initial_belief(pomdp)

updater = DiscreteUpdater(pomdp) # this comes from POMDPToolbox

rng = MersenneTwister(9) # initialize a random number generator

rtot = 0.0
# lets run the simulation for ten time steps
for i = 1:10
    # get the action from our SARSOP policy
    a = action(qmdp_policy, b) # the QMDP action function returns the POMDP action not its index like the SARSOP action function
    # compute the reward
    r = reward(pomdp, s, a)
    rtot += r
    
    println("Time step $i")
    println("Have belief: $(b.b), taking action: $(a), got reward: $(r)")
    
    # transition the system state
    trans_dist = transition(pomdp, s, a)
    rand(rng, trans_dist, s)

    # sample a new observation
    obs_dist = observation(pomdp, s, a)
    rand(rng, obs_dist, o)
    
    # update the belief
    b = update(updater, b, a, o)
    
    println("Saw observation: $(o), new belief: $(b.b)\n")

end
println("Total reward: $rtot")

Time step 1
Have belief: [0.5,0.5], taking action: TigerAction(:listen), got reward: -1.0
Saw observation: TigerObservation(true), new belief: [0.85,0.15000000000000002]

Time step 2
Have belief: [0.85,0.15000000000000002], taking action: TigerAction(:listen), got reward: -1.0
Saw observation: TigerObservation(true), new belief: [0.9697986577181208,0.030201342281879207]

Time step 3
Have belief: [0.9697986577181208,0.030201342281879207], taking action: TigerAction(:openr), got reward: 10.0
Saw observation: TigerObservation(false), new belief: [0.5,0.5]

Time step 4
Have belief: [0.5,0.5], taking action: TigerAction(:listen), got reward: -1.0
Saw observation: TigerObservation(false), new belief: [0.15000000000000002,0.85]

Time step 5
Have belief: [0.15000000000000002,0.85], taking action: TigerAction(:listen), got reward: -1.0
Saw observation: TigerObservation(false), new belief: [0.030201342281879207,0.9697986577181208]

Time step 6
Have belief: [0.030201342281879207,0.969798657718120

The results are identical! At least for this short simulation. In general, if you are dealing with a complex problem, it is good to compare the SARSOP and QMDP policies. This framework makes comparing the two policies very simple once you have defined the problem! 